In [1]:
import os
import cv2
import numpy as np
from PIL import Image
from tqdm import tqdm 

In [2]:
def create_color_to_label_map(mask_folder_path):
    """
    Scans all masks in a folder to find unique colors and create a mapping
    from color to a class label.
    """
    print(f"Scanning masks in '{mask_folder_path}'...")
    
    unique_colors = set()
    

    mask_files = os.listdir(mask_folder_path)
    print(mask_files)
   
    for filename in tqdm(mask_files, desc="Finding unique colors"):
        mask_path = os.path.join(mask_folder_path, filename)
        
        # Open the mask image and convert to a NumPy array
        mask_bgr = cv2.imread(mask_path)

        mask_rgb = cv2.cvtColor(mask_bgr, cv2.COLOR_BGR2RGB)
        # Reshape the (H, W, C) array to (H*W, C) to get a list of pixels
        pixels = mask_rgb.reshape(-1, 3)
        
        # Get unique rows (unique RGB colors)
        unique_pixel_colors = np.unique(pixels, axis=0)
        
        # Add the unique colors found in this image to our overall set
        for color in unique_pixel_colors:
            unique_colors.add(tuple(color))
            

    sorted_colors = sorted(list(unique_colors))
    
    # Create the color-to-label dictionary
    color_to_label = {color: label for label, color in enumerate(sorted_colors)}
    
    print("\nScan complete!")
    print(f"Found {len(color_to_label)} unique classes.")
    
    return color_to_label

In [3]:
mask_path = "./data_dataset_voc/SegmentationClass"
COLOR_TO_LABEL = create_color_to_label_map(mask_path)
COLOR_TO_LABEL

Scanning masks in './data_dataset_voc/SegmentationClass'...
['cat.6.png', 'cat.24.png', 'cat.30.png', 'cat.18.png', 'dog.22.png', 'dog.36.png', 'dog.37.png', 'dog.23.png', 'cat.19.png', 'cat.31.png', 'cat.25.png', 'cat.7.png', 'cat.5.png', 'cat.33.png', 'cat.27.png', 'dog.35.png', 'dog.21.png', 'dog.20.png', 'dog.34.png', 'cat.26.png', 'cat.32.png', 'cat.4.png', 'cat.0.png', 'cat.36.png', 'cat.22.png', 'dog.30.png', 'dog.24.png', 'dog.18.png', 'dog.19.png', 'dog.25.png', 'dog.31.png', 'cat.23.png', 'cat.37.png', 'cat.1.png', 'cat.3.png', 'cat.21.png', 'cat.35.png', 'dog.27.png', 'dog.33.png', 'dog.32.png', 'dog.26.png', 'cat.34.png', 'cat.20.png', 'cat.2.png', 'cat.47.png', 'cat.53.png', 'dog.82.png', 'cat.84.png', 'cat.90.png', 'dog.69.png', 'dog.1.png', 'dog.41.png', 'dog.55.png', 'dog.54.png', 'dog.40.png', 'dog.0.png', 'dog.68.png', 'cat.85.png', 'dog.83.png', 'cat.52.png', 'cat.46.png', 'cat.50.png', 'cat.44.png', 'cat.78.png', 'dog.81.png', 'cat.87.png', 'dog.56.png', 'dog.2.png'

Finding unique colors: 100%|██████████| 182/182 [00:18<00:00,  9.60it/s]


Scan complete!
Found 3 unique classes.


{(np.uint8(0), np.uint8(0), np.uint8(0)): 0,
 (np.uint8(0), np.uint8(128), np.uint8(0)): 1,
 (np.uint8(128), np.uint8(0), np.uint8(0)): 2}

In [4]:
import numpy as np
import cv2
from PIL import Image

def encode_mask_to_grayscale(mask_path, color_map):
    """
    Converts an RGB segmentation mask to a grayscale mask with class labels.
    """
    mask_bgr = cv2.imread(mask_path)
    mask_rgb = cv2.cvtColor(mask_bgr, cv2.COLOR_BGR2RGB)
    height, width, _ = mask_rgb.shape
    
    # Create an empty grayscale mask (height x width)
    mask_grayscale = np.zeros((height, width), dtype=np.uint8)
    
    # For each color in our map, find where it is in the mask and assign the label
    for color, label in color_map.items():
        # Find pixels matching the color
        matches = np.where(np.all(mask_rgb == color, axis=-1))
        mask_grayscale[matches] = label
        
    return mask_grayscale

In [13]:
import os
import cv2
import matplotlib.pyplot as plt

# Define the paths for input and output directories
mask_path_dir = "./data_dataset_voc/SegmentationClass/"
save_path_dir = "./data_dataset_voc/encoded_masks/"

# Loop through all files in the segmentation class directory
for filename in os.listdir(mask_path_dir):
    # Process only .png files to avoid hidden files or other formats
    if filename.endswith(".png"):
        mask_path = os.path.join(mask_path_dir, filename)
        
        # Construct the full path for the output file
        # The output file will have the same name but can have a different extension if desired
        base_filename = os.path.splitext(filename)[0]
        save_path = os.path.join(save_path_dir, f"{base_filename}.png")
        
        print(f"Processing {filename}...")
        
        # Convert the mask to grayscale
        grayscale_label_mask = encode_mask_to_grayscale(mask_path, COLOR_TO_LABEL)
        
        # Save the grayscale mask
        cv2.imwrite(save_path, grayscale_label_mask)
        
print("\nAll masks have been converted and saved.")

Processing cat.6.png...
Processing cat.24.png...
Processing cat.30.png...
Processing cat.18.png...
Processing dog.22.png...
Processing dog.36.png...
Processing dog.37.png...
Processing dog.23.png...
Processing cat.19.png...
Processing cat.31.png...
Processing cat.25.png...
Processing cat.7.png...
Processing cat.5.png...
Processing cat.33.png...
Processing cat.27.png...
Processing dog.35.png...
Processing dog.21.png...
Processing dog.20.png...
Processing dog.34.png...
Processing cat.26.png...
Processing cat.32.png...
Processing cat.4.png...
Processing cat.0.png...
Processing cat.36.png...
Processing cat.22.png...
Processing dog.30.png...
Processing dog.24.png...
Processing dog.18.png...
Processing dog.19.png...
Processing dog.25.png...
Processing dog.31.png...
Processing cat.23.png...
Processing cat.37.png...
Processing cat.1.png...
Processing cat.3.png...
Processing cat.21.png...
Processing cat.35.png...
Processing dog.27.png...
Processing dog.33.png...
Processing dog.32.png...
Process